<a href="https://colab.research.google.com/github/sid-betalol/AmEx-Credit-Card-Default-Prediction/blob/main/notebooks/N00_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and Installations

In [ ]:
!pip install rdkit
!pip install pypdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 44.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.AtomPairs import Torsions
from rdkit.Chem.Fingerprints import FingerprintMols

In [ ]:
import os
import sys
import os.path
from sys import platform
from pathlib import Path

In [ ]:
import ast
import copy
import time
import scipy
import random
import pickle
import scipy.io
import argparse
import subprocess
import numpy as np
import pandas as pd

In [ ]:
from numpy import *
from tqdm import tqdm
from pathlib import Path
from random import shuffle

In [ ]:
from pypdb import *

Dataframe

In [ ]:
def beautiful_print(df):
    # Print the dataset in a well-organized format.
    with pd.option_context('display.max_rows', 20,
                           'display.min_rows', 20,
                           'display.max_columns', 6,
                           #"display.max_colwidth", None,
                           "display.width", None,
                           "expand_frame_repr", True,
                           "max_seq_items", None,):  # more options can be specified
        # Once the display.max_rows is exceeded,
        # the display.min_rows options determines
        # how many rows are shown in the truncated repr.
        print(df)
    return

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def Get_processed_data_df(data_folder, data_file, data_file_binary, binary_class_bool, y_prpty_cls_threshold, target_nme, max_seqs_len):
    #====================================================================================================#
    processed_data_df = pd.read_csv(data_folder / data_file, index_col=0, header=0)


    if type(target_nme) == str:
        processed_data_df_1 = processed_data_df.groupby('SEQ').agg(
                                val_avg = pd.NamedAgg(column = target_nme, aggfunc = 'mean') ,
                                val_SD  = pd.NamedAgg(column = target_nme, aggfunc = 'std' ) ,
                                                                                            )
        #beautiful_print(processed_data_df_1)
        processed_data_df_1["val_SD"] = processed_data_df_1["val_SD"].fillna(value=0)
        processed_data_df_1.reset_index(inplace = True)
        processed_data_df_2 = processed_data_df_1[ np.isnan(processed_data_df_1['val_avg']) == False]
        #beautiful_print(processed_data_df_2)
        processed_data_df = pd.merge(processed_data_df, processed_data_df_2, on = ['SEQ'], how = 'left')
        processed_data_df.drop(columns = [target_nme, ], inplace = True)
        processed_data_df = processed_data_df.dropna(subset = ['val_avg'])
        processed_data_df.rename(columns={'val_avg': target_nme}, inplace = True)
        processed_data_df.drop_duplicates(subset = ['SEQ'], keep = 'first', inplace = True)
        processed_data_df.reset_index(inplace = True)
        processed_data_df = processed_data_df[['SEQ', target_nme]]
        #beautiful_print(processed_data_df)



    # Remove sequences that are too long (remove those longer than max_seqs_len)
    processed_data_df["seqs_length"] = processed_data_df.SEQ.str.len()
    processed_data_df = processed_data_df[processed_data_df.seqs_length <= max_seqs_len]
    processed_data_df.reset_index(drop = True, inplace = True)

    #====================================================================================================#
    # Current version doesnt support binary classifica
    if binary_class_bool and ((data_folder / data_file_binary).exists()):
        processed_data_df_bi = pd.read_csv(data_folder / data_file_binary, index_col=0, header=0)

    else:
        print("binary classification file does not exist.")
        processed_data_df_bi = pd.read_csv(data_folder / data_file, index_col=0, header=0)

        if type(target_nme) == dict or type(target_nme) == list:
            for one_target in target_nme:
                processed_data_df_bi[one_target] = [1 if one_cvsn>y_prpty_cls_threshold else 0
                                                      for one_cvsn in list(processed_data_df_bi[one_target])]
        elif type(target_nme) == str:
            processed_data_df_bi[target_nme] = [1 if one_cvsn>y_prpty_cls_threshold else 0
                                                  for one_cvsn in list(processed_data_df_bi[target_nme])]
    #--------------------------------------------------#
    # Remove sequences that are too long (remove those longer than max_seqs_len)
    processed_data_df_bi["seqs_length"] = processed_data_df_bi.SEQ.str.len()
    processed_data_df_bi = processed_data_df_bi[processed_data_df_bi.seqs_length <= max_seqs_len]
    processed_data_df_bi.reset_index(drop = True, inplace = True)


    return processed_data_df, processed_data_df_bi

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Write FASTA

In [ ]:
def output_fasta(processed_data_df, output_folder, output_file_0):
    # Also get a seqs_list including all sequences
    processed_data_df_row_num = processed_data_df.shape[0]
    with open(output_folder / output_file_0 , 'w') as f:
        count_x=0
        seqs_list=[]
        max_len = 0
        print("processed_data_df_row_num: ", processed_data_df_row_num)
        for i in range(processed_data_df_row_num):
            one_seq = (processed_data_df.loc[i,"SEQ"]).replace("-", "")
            max_len = len(one_seq) if len(one_seq)>max_len else max_len
            if one_seq not in seqs_list and len(one_seq)<=max_seqs_len:
                seqs_list.append(one_seq)
                count_x+=1
                if len(one_seq) <= 1024-2:
                    f.write(">seq"+str(count_x)+"\n")
                    f.write(one_seq.upper()+"\n")
                else:
                    f.write(">seq"+str(count_x)+"\n")
                    f.write(one_seq.upper()[0 : 1024-2]+"\n")
    print("number of seqs: ", len(seqs_list))
    print("number of seqs duplicates removed: ", len(set(seqs_list)))
    return seqs_list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Outputs

In [ ]:
def output_formated_dataset(processed_data_df,
                            processed_data_df_bi  ,
                            seqs_list             ,
                            y_prpty_cls_threshold ,
                            target_nme            ,
                            data_folder           ,
                            output_folder         ,
                            output_file_1         ,
                            ):
    #--------------------------------------------------#
    # Initialize.
    seq_prpty_dict = dict([])
    #--------------------------------------------------#
    #
    if type(target_nme) == dict or type(target_nme) == list:
        seq_prpty_dict["sequence"] = seqs_list
        for one_header in target_nme:
            seq_prpty_dict[one_header] = processed_data_df[one_header]

    #print(np.isnan(processed_data_df.loc[17, "FC1"]))
    #print(seq_prpty_dict["FC1"][18])

    pickle.dump( seq_prpty_dict, open( output_folder / output_file_1, "wb" ) )

    return seq_prpty_dict

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Data Processing

In [ ]:
def N00_Data_Processing(binary_class_bool      ,
                        data_folder            ,
                        target_nme             ,
                        data_file              ,
                        data_file_binary       ,
                        max_seqs_len           ,
                        y_prpty_cls_threshold  ,
                        output_folder          ,
                        output_file_0          ,
                        output_file_1          ,
                        ):
    # Process target_nme



    # Get_processed_data_df
    processed_data_df, processed_data_df_bi = Get_processed_data_df(data_folder           ,
                                                                    data_file             ,
                                                                    data_file_binary      ,
                                                                    binary_class_bool     ,
                                                                    y_prpty_cls_threshold ,
                                                                    target_nme            ,
                                                                    max_seqs_len          ,
                                                                    )
    beautiful_print(processed_data_df)
    beautiful_print(processed_data_df_bi)

    #--------------------------------------------------#
    # Output #1: Write a fasta file including all sequences
    seqs_list = output_fasta(processed_data_df, output_folder, output_file_0)

    #--------------------------------------------------#
    # Output #2: Write compounds_properties_list
    seqs_prpty_dict = output_formated_dataset(processed_data_df     ,
                                              processed_data_df_bi  ,
                                              seqs_list             ,
                                              y_prpty_cls_threshold ,
                                              target_nme            ,
                                              data_folder           ,
                                              output_folder         ,
                                              output_file_1         ,
                                              )
    #--------------------------------------------------#

    return

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Preprocessing the dataset

In [ ]:
Step_code = "N00_"
dataset_nme = "GFP"
data_info_dict = {"GFP": ["quantitative_function"  , "./subset_gfp_10000.csv", 1200, ]}
target_nme   = data_info_dict[dataset_nme][0]
data_file    = data_info_dict[dataset_nme][1]
max_seqs_len = data_info_dict[dataset_nme][2]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
if target_nme in locals().keys():
    target_nme = locals()[target_nme]
else:
    locals()[target_nme] = {target_nme : target_nme}
    target_nme = locals()[target_nme]
print(target_nme)

{'quantitative_function': 'quantitative_function'}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
binary_class_bool = True

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_folder = Path("/content/GFP/")
data_file_binary = data_file.replace(".csv", "_binary.csv") # y_prpty_cls_threshold = around 1e-2
y_prpty_cls_threshold = 1e-5 # Used for type II screening
output_folder = Path("/content/N_DataProcessed")
output_file_0 = Step_code + dataset_nme + ".fasta"
output_file_1 = Step_code + dataset_nme + "_seqs_prpty_list.p"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Preprocessing call

In [ ]:
N00_Data_Processing(binary_class_bool, data_folder, target_nme, data_file, data_file_binary, max_seqs_len, y_prpty_cls_threshold, output_folder, output_file_0, output_file_1)
print("*" * 50)
print(Step_code + " Done!")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


binary classification file does not exist.
                                                    SEQ  \
0     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
1     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
2     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
3     MSKGEELFTGVVLILVEPDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
4     MSKGEELFTGVVSIPVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
5     MSKGEELLTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
6     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
7     MSKGEELFTGVVPILVELDGDVNGHKFSVSGVGEGDATNGKLTLKF...   
8     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
9     MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
...                                                 ...   
9990  MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
9991  MSKGEELFTGVVPILVELDGDVNGHKLSVSGEGEGDATYGKLTLKF...   
9992  MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
9993  MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...   
9994  MSKGEGL